# Homework 5: Oliver Muellerklein

Collaboration with Jacob Bukoski and Zhongqi Miao.

In [4]:
import sys

#sys.path.append('/usr/local/lib/python3.5/site-packages')
#sys.path

### Title Here:

Some description:

* dfdf...

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def amdahl(s, p):
    return 1.0 / (s + (1.0 - s) / p)

p = np.logspace(0, 16, 9, base = 2)
#ax = plt.subplot(111)

for s in [0.5, 0.25, 0.1, 0.05]:
    sp = amdahl(s, p)
    #ax.semilogx(p, sp, '-o', label = 's = %.2f' % s, basex = 2.0)

#ax.set_xlabel('Processors')
#ax.set_ylabel('Speedup')
#ax.set_title('Speedup as a function of serial fraction')
#ax.legend()
#ax.grid()
#plt.show()

Words here...

In [1]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
    
response = urlopen("http://words.bighugelabs.com/")
html = response.read()
response.close()

# pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html5lib")
forms = soup.findAll("form")
#forms

In [51]:
response08 = urlopen("https://www.wunderground.com/history/airport/%s/%s/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=%s" \
                   % ('KATL', '2008', '2008'))
html08 = response08.read()
response08.close()
soup08 = BeautifulSoup(html08, "html5lib")

#tbody = soup.find('table', id = 'obsTable', class_ = 'responsive obs-table daily').find_all('tbody')

In [12]:
#tBody = soup.find("table", id = "obsTable", class_ = "reponsive obs-table daily").find_all("tbody")

tbody08 = soup08.find('table', id = 'obsTable', class_ = 'responsive obs-table daily').find_all('tbody')

In [48]:
monthIndx = []

#feb_td08 = tbody08[32].find_all('td')

def mapSwitcher(argument):
    
    switchh = {
        'Jan' : True,
        'Feb' : True
    }
    
    return switchh.get(argument, 'nothing')

global counterr
counterr = 0

#jan_td08 = tbody08[counterr].find_all('td')[0].string


In [49]:

def trySwitcher(tryTd):

    #if (mapSwitcher(jan_td08)):
    if (mapSwitcher(tryTd)):
        # Get index from tbody 
        monthIndx.append(counterr)
        global counterr
        counterr += 32
        return False

    else:
        # Try again 
        global counterr
        counterr += 1
        return True
    
        #jan_td08 = tbody08[counterr].find_all('td')[0].string
        #
        #if (mapSwitcher(jan_td08)):
        ## Get index from tbody 
        #monthIndx.append(counterr)
        #counterr += 32
        
jan_td08 = tbody08[counterr].find_all('td')[0].string
tryIt = trySwitcher(jan_td08)
        
if (tryIt):
    jan_td08 = tbody08[counterr].find_all('td')[0].string
    tryIt = trySwitcher(jan_td08)

<ipython-input-49-fc5dc867ce16>:8: SyntaxWarning: name 'counterr' is used prior to global declaration
  global counterr
<ipython-input-49-fc5dc867ce16>:14: SyntaxWarning: name 'counterr' is assigned to before global declaration
  global counterr


In [50]:
counterr

32

Words here...